In [ ]:
import socket
import struct

# Convert IP to integer
def ip_to_int(ip):
    return struct.unpack("!I", socket.inet_aton(ip))[0]

# Convert IP addresses to integers in fraud_data
fraud_data["ip_int"] = fraud_data["ip_address"].apply(ip_to_int)

# Load and process ip-to-country data
ip_country = pd.read_csv("../data/IpAddress_to_Country.csv")
ip_country["lower_bound_ip_int"] = ip_country["lower_bound_ip_address"].apply(ip_to_int)
ip_country["upper_bound_ip_int"] = ip_country["upper_bound_ip_address"].apply(ip_to_int)

# Sort both dataframes for merge_asof
fraud_data_sorted = fraud_data.sort_values("ip_int")
ip_country_sorted = ip_country.sort_values("lower_bound_ip_int")

# Use merge_asof to assign country
merged = pd.merge_asof(
    fraud_data_sorted,
    ip_country_sorted,
    left_on="ip_int",
    right_on="lower_bound_ip_int",
    direction="backward"
)

# Filter where ip_int is within bounds
merged = merged[merged["ip_int"] <= merged["upper_bound_ip_int"]]

# Preview
merged[["ip_address", "ip_int", "country"]].head()
